In [24]:
import pandas as pd
import numpy as np

In [26]:
df = pd.read_pickle("../data/data_final.pkl")

labels = ["episode_type","trial_type","trial_num",
          "episode_num","is_corrupt","action_index",
          "trimmed_data","left","subject_num","win"]

df.describe()

,0,1,2,3,4,5,6,7,8,9,...,episode_type,episode_num,action_index,trimmed_data,is_corrupt,trial_type,trial_num,subject_num,win,left
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,...,1161.000000,1161.000000,1161.000000,1161.000000,1161.0,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,52.022394,53.587425,53.253230,54.522825,54.908699,55.260121,55.435831,55.204996,54.273040,53.268734,...,1.551249,14.476314,1025.596038,34.442722,0.0,0.485788,19.989664,5.433247,0.616710,0.478898
std,81.010447,82.020768,89.306357,89.356674,91.521565,95.338388,95.459503,95.162054,94.044461,94.112278,...,1.217403,8.650274,8.674554,8.294080,0.0,0.500013,11.250589,2.878139,0.486398,0.499770
min,-409.000000,-406.000000,-1014.000000,-1027.000000,-1064.000000,-1161.000000,-1210.000000,-1178.000000,-1169.000000,-1195.000000,...,0.000000,0.000000,987.000000,1.000000,0.0,0.000000,1.000000,1.000000,0.000000,0.000000
25%,2.000000,12.000000,12.000000,19.000000,18.000000,18.000000,18.000000,20.000000,21.000000,20.000000,...,0.000000,7.000000,1024.000000,34.000000,0.0,0.000000,10.000000,3.000000,0.000000,0.000000
50%,54.000000,55.000000,56.000000,57.000000,57.000000,56.000000,57.000000,57.000000,57.000000,56.000000,...,2.000000,14.000000,1025.000000,34.000000,0.0,0.000000,20.000000,5.000000,1.000000,0.000000
75%,85.000000,85.000000,86.000000,87.000000,89.000000,90.000000,89.000000,87.000000,88.000000,84.000000,...,3.000000,22.000000,1025.000000,38.000000,0.0,1.000000,30.000000,8.000000,1.000000,1.000000
max,699.000000,707.000000,696.000000,668.000000,651.000000,695.000000,715.000000,707.000000,740.000000,747.000000,...,3.000000,29.000000,1062.000000,76.000000,0.0,1.000000,39.000000,10.000000,1.000000,1.000000


### resample the eeg readings at a lower frequency

In [28]:
resamp_frequency = 100 #Hz
crop_stride = int(512/resamp_frequency)

#break into eeg readings and labels and create placeholder dataframe for resampled data 
eeg_df, labels_df = df[list(range(1880))], df[labels]
eeg_df_resampled = pd.DataFrame()

#for each episode in original eeg_df calculate the moving average along the entire row of readings
for r in range(eeg_df.shape[0]):
    #get the readings for the rth episode
    row = eeg_df.iloc[r]
    row_ = []
    #calculate moving average every crop_stride readings and append to row_
    for c in range(crop_stride, eeg_df.shape[1], crop_stride):
        row_.append(np.mean(row[c-crop_stride: c]))
    
    #append resampled version of episode's eeg data to placeholder df
    eeg_df_resampled = eeg_df_resampled.append([row_],ignore_index=True)

#join resampled eeg readings with the original labels
df_ = eeg_df_resampled.join(labels_df)
df_.describe()

,0,1,2,3,4,5,6,7,8,9,...,episode_type,trial_type,trial_num,episode_num,is_corrupt,action_index,trimmed_data,left,subject_num,win
count,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000,...,1161.000000,1161.000000,1161.000000,1161.000000,1161.0,1161.000000,1161.000000,1161.000000,1161.000000,1161.000000
mean,53.658915,54.688544,54.459087,54.496985,51.034281,51.154522,50.852713,53.579500,56.256331,59.561929,...,1.551249,0.485788,19.989664,14.476314,0.0,1025.596038,34.442722,0.478898,5.433247,0.616710
std,82.088600,92.533246,89.430760,84.986394,78.625794,73.616756,71.184671,71.539571,73.862749,77.954870,...,1.217403,0.500013,11.250589,8.650274,0.0,8.674554,8.294080,0.499770,2.878139,0.486398
min,-621.000000,-1182.600000,-1100.600000,-890.000000,-511.200000,-387.400000,-391.400000,-397.600000,-386.200000,-409.400000,...,0.000000,0.000000,1.000000,0.000000,0.0,987.000000,1.000000,0.000000,1.000000,0.000000
25%,20.600000,24.000000,30.200000,33.600000,31.400000,30.600000,31.800000,31.000000,33.800000,36.600000,...,0.000000,0.000000,10.000000,7.000000,0.0,1024.000000,34.000000,0.000000,3.000000,0.000000
50%,56.400000,57.400000,57.400000,58.400000,54.200000,55.400000,55.400000,58.200000,57.600000,60.000000,...,2.000000,0.000000,20.000000,14.000000,0.0,1025.000000,34.000000,0.000000,5.000000,1.000000
75%,84.200000,85.800000,83.200000,83.600000,80.600000,81.200000,80.000000,82.800000,83.600000,82.600000,...,3.000000,1.000000,30.000000,22.000000,0.0,1025.000000,38.000000,1.000000,8.000000,1.000000
max,681.200000,690.000000,675.200000,517.800000,434.400000,561.000000,648.200000,634.000000,510.800000,536.600000,...,3.000000,1.000000,39.000000,29.000000,0.0,1062.000000,76.000000,1.000000,10.000000,1.000000


In [29]:
# df_.to_pickle("../data/data_100hz.pkl")